Import MyDataBase class.

In [1]:
from src.database import MyDataBase
db_name = "parch_and_posey"

Load database.

In [2]:
database = MyDataBase(db_name)
database.print_tables_names()

ISOLATION_LEVEL_AUTOCOMMIT: 0
('web_events',)
('sales_reps',)
('region',)
('orders',)
('accounts',)


### LEFT & RIGHT Quizzes

1. In the accounts table, there is a column holding the website for each company.
The last three digits specify what type of web address they are using.
A list of extensions (and pricing) is provided [here](https://iwantmyname.com/domains).
Pull these extensions and provide how many of each website type exist in the accounts table.

In [3]:
q = """
SELECT RIGHT(website, 3) AS domain, COUNT(*) num_companies
FROM accounts
GROUP BY 1
ORDER BY 2 DESC;
"""

table = database.to_data_frame(q)
table

,domain,num_companies
0,com,349
1,net,1
2,org,1


2. There is much debate about how much the name (or even the first letter of a company name) matters.
Use the accounts table to pull the first letter of each company name
to see the distribution of company names that begin with each letter (or number).

In [4]:
q = """
SELECT LEFT(UPPER(name), 1) AS first_letter, COUNT(*) num_companies
FROM accounts
GROUP BY 1
ORDER BY 2 DESC;
"""

table = database.to_data_frame(q)
table

,first_letter,num_companies
0,C,37
1,A,37
2,P,27
3,M,22
4,S,17
5,D,17
6,T,17
7,B,16
8,E,16
9,L,16


3. Use the accounts table and a CASE statement to create two groups:
one group of company names that start with a number
and a second group of those company names that start with a letter.
What proportion of company names start with a letter?

In [5]:
q = """
SELECT SUM(num) nums, SUM(letter) letters
FROM (SELECT name, CASE WHEN LEFT(UPPER(name), 1) IN ('0','1','2','3','4','5','6','7','8','9')
                       THEN 1 ELSE 0 END AS num,
         CASE WHEN LEFT(UPPER(name), 1) IN ('0','1','2','3','4','5','6','7','8','9')
                       THEN 0 ELSE 1 END AS letter
      FROM accounts) t1;
"""

table = database.to_data_frame(q)
table

,nums,letters
0,1,350


There are 350 company names that start with a letter and 1 that starts with a number. This gives a ratio of 350/351 that are company names that start with a letter or 99.7%.

4. Consider vowels as a, e, i, o, and u.
What proportion of company names start with a vowel, and what percent start with anything else?

In [6]:
q = """
SELECT SUM(vowels) vowels, SUM(other) other
FROM (SELECT name, CASE WHEN LEFT(UPPER(name), 1) IN ('A','E','I','O','U')
                        THEN 1 ELSE 0 END AS vowels,
          CASE WHEN LEFT(UPPER(name), 1) IN ('A','E','I','O','U')
                       THEN 0 ELSE 1 END AS other
         FROM accounts) t1;
"""

table = database.to_data_frame(q)
table

,vowels,other
0,80,271


There are 80 company names that start with a vowel and 271 that start with other characters.
Therefore 80/351 are vowels or 22.8%. Therefore, 77.2% of company names do not start with vowels.


### Quizzes POSITION & STRPOS


1. Use the accounts table to create first and last name columns
that hold the first and last names for the primary_poc.

In [7]:
q = """
SELECT LEFT(primary_poc, STRPOS(primary_poc, ' ') -1 ) first_name,
RIGHT(primary_poc, LENGTH(primary_poc) - STRPOS(primary_poc, ' ')) last_name
FROM accounts;
"""

table = database.to_data_frame(q)
table

,first_name,last_name
0,Tamara,Tuma
1,Sung,Shields
2,Jodee,Lupo
3,Serafina,Banda
4,Angeles,Crusoe
...,...,...
346,Buffy,Azure
347,Esta,Engelhardt
348,Khadijah,Riemann
349,Deanne,Hertlein


2. Now see if you can do the same thing for every rep name in the sales_reps table.
Again provide first and last name columns.

In [8]:
q = """
SELECT LEFT(name, STRPOS(name, ' ') -1 ) first_name,
       RIGHT(name, LENGTH(name) - STRPOS(name, ' ')) last_name
FROM sales_reps;
"""

table = database.to_data_frame(q)
table

,first_name,last_name
0,Samuel,Racine
1,Eugena,Esser
2,Michel,Averette
3,Renetta,Carew
4,Cara,Clarke
5,Lavera,Oles
6,Elba,Felder
7,Shawanda,Selke
8,Sibyl,Lauria
9,Necole,Victory


### Concat examples

1. Each company in the accounts table wants to create an email address for each primary_poc.
The email address should be
the first name of the primary_poc "." last name primary_poc @ company name .com.

In [10]:
q = """
WITH t1 AS (
 SELECT LEFT(primary_poc, STRPOS(primary_poc, ' ') -1 ) first_name,  RIGHT(primary_poc, LENGTH(primary_poc) - STRPOS(primary_poc, ' ')) last_name, name
 FROM accounts)
SELECT first_name, last_name, CONCAT(first_name, '.', last_name, '@', name, '.com')
FROM t1;
"""

table = database.to_data_frame(q)
table

,first_name,last_name,concat
0,Tamara,Tuma,Tamara.Tuma@Walmart.com
1,Sung,Shields,Sung.Shields@Exxon Mobil.com
2,Jodee,Lupo,Jodee.Lupo@Apple.com
3,Serafina,Banda,Serafina.Banda@Berkshire Hathaway.com
4,Angeles,Crusoe,Angeles.Crusoe@McKesson.com
...,...,...,...
346,Buffy,Azure,Buffy.Azure@KKR.com
347,Esta,Engelhardt,Esta.Engelhardt@Oneok.com
348,Khadijah,Riemann,Khadijah.Riemann@Newmont Mining.com
349,Deanne,Hertlein,Deanne.Hertlein@PPL.com


2. You may have noticed that in the previous solution some of the company names include spaces,
which will certainly not work in an email address.
See if you can create an email address that will work by removing all of the spaces
in the account name, but otherwise your solution should be just as in question 1.
Some helpful documentation is [here](https://www.postgresql.org/docs/8.1/functions-string.html).

In [11]:
q = """
WITH t1 AS (
 SELECT LEFT(primary_poc, STRPOS(primary_poc, ' ') -1 ) first_name,  RIGHT(primary_poc, LENGTH(primary_poc) - STRPOS(primary_poc, ' ')) last_name, name
 FROM accounts)
SELECT first_name, last_name, CONCAT(first_name, '.', last_name, '@', REPLACE(name, ' ', ''), '.com')
FROM  t1;
"""

table = database.to_data_frame(q)
table

,first_name,last_name,concat
0,Tamara,Tuma,Tamara.Tuma@Walmart.com
1,Sung,Shields,Sung.Shields@ExxonMobil.com
2,Jodee,Lupo,Jodee.Lupo@Apple.com
3,Serafina,Banda,Serafina.Banda@BerkshireHathaway.com
4,Angeles,Crusoe,Angeles.Crusoe@McKesson.com
...,...,...,...
346,Buffy,Azure,Buffy.Azure@KKR.com
347,Esta,Engelhardt,Esta.Engelhardt@Oneok.com
348,Khadijah,Riemann,Khadijah.Riemann@NewmontMining.com
349,Deanne,Hertlein,Deanne.Hertlein@PPL.com


3. We would also like to create an initial password,
which they will change after their first log in.
The first password will be the first letter of the primary_poc's first name (lowercase),
then the last letter of their first name (lowercase),
the first letter of their last name (lowercase),
the last letter of their last name (lowercase),
the number of letters in their first name,
the number of letters in their last name,
and then the name of the company they are working with,
all capitalized with no spaces.

In [14]:
q = """
WITH t1 AS (
 SELECT LEFT(primary_poc,     STRPOS(primary_poc, ' ') -1 ) first_name,  RIGHT(primary_poc, LENGTH(primary_poc) - STRPOS(primary_poc, ' ')) last_name, name
 FROM accounts)
SELECT first_name, last_name, CONCAT(first_name, '.', last_name, '@', name, '.com') as email,
    LEFT(LOWER(first_name), 1) || RIGHT(LOWER(first_name), 1) || LEFT(LOWER(last_name), 1) || RIGHT(LOWER(last_name), 1) || LENGTH(first_name) || LENGTH(last_name) || REPLACE(UPPER(name), ' ', '') as password
FROM t1;
"""

table = database.to_data_frame(q)
table

,first_name,last_name,email,password
0,Tamara,Tuma,Tamara.Tuma@Walmart.com,tata64WALMART
1,Sung,Shields,Sung.Shields@Exxon Mobil.com,sgss47EXXONMOBIL
2,Jodee,Lupo,Jodee.Lupo@Apple.com,jelo54APPLE
3,Serafina,Banda,Serafina.Banda@Berkshire Hathaway.com,saba85BERKSHIREHATHAWAY
4,Angeles,Crusoe,Angeles.Crusoe@McKesson.com,asce76MCKESSON
...,...,...,...,...
346,Buffy,Azure,Buffy.Azure@KKR.com,byae55KKR
347,Esta,Engelhardt,Esta.Engelhardt@Oneok.com,eaet410ONEOK
348,Khadijah,Riemann,Khadijah.Riemann@Newmont Mining.com,khrn87NEWMONTMINING
349,Deanne,Hertlein,Deanne.Hertlein@PPL.com,dehn68PPL


### Cast examples

No examples are given as all of the data in this particular dataset were already clean.

###  Coalesce

1. Run the query below to notice the row with missing data.

In [15]:
q = """
SELECT *
FROM accounts a
LEFT JOIN orders o
ON a.id = o.account_id
WHERE o.total IS NULL;
"""

table = database.to_data_frame(q)
table

,id,name,website,lat,long,primary_poc,sales_rep_id,id,account_id,occurred_at,standard_qty,gloss_qty,poster_qty,total,standard_amt_usd,gloss_amt_usd,poster_amt_usd,total_amt_usd
0,1731,Goldman Sachs Group,www.gs.com,40.757444,-73.967309,Loris Manfredi,321690,None,None,None,None,None,None,None,None,None,None,None


2. Use COALESCE to fill in the accounts.id column
with the account.id for the NULL value for the table in 1.

In [16]:
q = """
SELECT COALESCE(a.id, a.id) filled_id, a.name, a.website, a.lat, a.long, a.primary_poc, a.sales_rep_id, o.*
FROM accounts a
LEFT JOIN orders o
ON a.id = o.account_id
WHERE o.total IS NULL;
"""

table = database.to_data_frame(q)
table

,filled_id,name,website,lat,long,primary_poc,sales_rep_id,id,account_id,occurred_at,standard_qty,gloss_qty,poster_qty,total,standard_amt_usd,gloss_amt_usd,poster_amt_usd,total_amt_usd
0,1731,Goldman Sachs Group,www.gs.com,40.757444,-73.967309,Loris Manfredi,321690,None,None,None,None,None,None,None,None,None,None,None


3. Use COALESCE to fill in the orders.account_id column
with the account.id for the NULL value for the table in 1.

In [17]:
q = """
SELECT
  COALESCE(a.id, a.id) filled_id, a.name, a.website, a.lat, a.long, a.primary_poc, a.sales_rep_id,
  COALESCE(o.account_id, a.id) account_id, o.occurred_at, o.standard_qty, o.gloss_qty, o.poster_qty, o.total, o.standard_amt_usd, o.gloss_amt_usd, o.poster_amt_usd, o.total_amt_usd
FROM accounts a
LEFT JOIN orders o
ON a.id = o.account_id
WHERE o.total IS NULL;
"""

table = database.to_data_frame(q)
table

,filled_id,name,website,lat,long,primary_poc,sales_rep_id,account_id,occurred_at,standard_qty,gloss_qty,poster_qty,total,standard_amt_usd,gloss_amt_usd,poster_amt_usd,total_amt_usd
0,1731,Goldman Sachs Group,www.gs.com,40.757444,-73.967309,Loris Manfredi,321690,1731,None,None,None,None,None,None,None,None,None


4. Use COALESCE to fill in each of the qty and usd column with 0 for table in 1.

In [18]:
q = """
SELECT
  COALESCE(a.id, a.id) filled_id, a.name, a.website, a.lat, a.long, a.primary_poc, a.sales_rep_id,
  COALESCE(o.account_id, a.id) account_id, o.occurred_at, COALESCE(o.standard_qty, 0) standard_qty,
  COALESCE(o.gloss_qty,0) gloss_qty, COALESCE(o.poster_qty,0) poster_qty, COALESCE(o.total,0) total,
  COALESCE(o.standard_amt_usd,0) standard_amt_usd, COALESCE(o.gloss_amt_usd,0) gloss_amt_usd,
  COALESCE(o.poster_amt_usd,0) poster_amt_usd, COALESCE(o.total_amt_usd,0) total_amt_usd
FROM accounts a
LEFT JOIN orders o
ON a.id = o.account_id
WHERE o.total IS NULL;
"""

table = database.to_data_frame(q)
table

,filled_id,name,website,lat,long,primary_poc,sales_rep_id,account_id,occurred_at,standard_qty,gloss_qty,poster_qty,total,standard_amt_usd,gloss_amt_usd,poster_amt_usd,total_amt_usd
0,1731,Goldman Sachs Group,www.gs.com,40.757444,-73.967309,Loris Manfredi,321690,1731,None,0,0,0,0,0.0,0.0,0.0,0.0


5. Run the query in 1 with the WHERE removed and COUNT
the number of ids.

In [19]:
q = """
SELECT COUNT(*)
FROM accounts a
LEFT JOIN orders o
ON a.id = o.account_id;
"""

table = database.to_data_frame(q)
table

,count
0,6913


6. Run the query in 5, but with the COALESCE function
used in questions 2 through 4.

In [22]:
q = """
SELECT
COALESCE(a.id, a.id) filled_id, a.name, a.website, a.lat, a.long, a.primary_poc, a.sales_rep_id,
COALESCE(o.account_id, a.id) account_id, o.occurred_at,
COALESCE(o.standard_qty, 0) standard_qty,
COALESCE(o.gloss_qty,0) gloss_qty,
COALESCE(o.poster_qty,0) poster_qty,
COALESCE(o.total,0) total,
COALESCE(o.standard_amt_usd,0) standard_amt_usd,
COALESCE(o.gloss_amt_usd,0) gloss_amt_usd,
COALESCE(o.poster_amt_usd,0) poster_amt_usd,
COALESCE(o.total_amt_usd,0) total_amt_usd
FROM accounts a
LEFT JOIN orders o
ON a.id = o.account_id;
"""

table = database.to_data_frame(q)
table

,filled_id,name,website,lat,long,primary_poc,sales_rep_id,account_id,occurred_at,standard_qty,gloss_qty,poster_qty,total,standard_amt_usd,gloss_amt_usd,poster_amt_usd,total_amt_usd
0,1001,Walmart,www.walmart.com,40.238496,-75.103297,Tamara Tuma,321500,1001,2015-10-06 17:31:14,123,22,24,169,613.77,164.78,194.88,973.43
1,1001,Walmart,www.walmart.com,40.238496,-75.103297,Tamara Tuma,321500,1001,2015-11-05 03:34:33,190,41,57,288,948.10,307.09,462.84,1718.03
2,1001,Walmart,www.walmart.com,40.238496,-75.103297,Tamara Tuma,321500,1001,2015-12-04 04:21:55,85,47,0,132,424.15,352.03,0.00,776.18
3,1001,Walmart,www.walmart.com,40.238496,-75.103297,Tamara Tuma,321500,1001,2016-01-02 01:18:24,144,32,0,176,718.56,239.68,0.00,958.24
4,1001,Walmart,www.walmart.com,40.238496,-75.103297,Tamara Tuma,321500,1001,2016-02-01 19:27:27,108,29,28,165,538.92,217.21,227.36,983.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6908,4501,SpartanNash,www.spartannash.com,45.555651,-122.657145,Jewell Likes,321970,4501,2016-07-29 19:58:32,5,91,96,192,24.95,681.59,779.52,1486.06
6909,4501,SpartanNash,www.spartannash.com,45.555651,-122.657145,Jewell Likes,321970,4501,2016-08-27 00:58:11,16,94,82,192,79.84,704.06,665.84,1449.74
6910,4501,SpartanNash,www.spartannash.com,45.555651,-122.657145,Jewell Likes,321970,4501,2016-11-22 06:52:22,63,67,81,211,314.37,501.83,657.72,1473.92
6911,4501,SpartanNash,www.spartannash.com,45.555651,-122.657145,Jewell Likes,321970,4501,2016-12-21 13:30:42,61,150,52,263,304.39,1123.50,422.24,1850.13


In [23]:
database.close()
